**Notebook diagnostic:** Run the next code cell to show which Python executable this notebook kernel is using and whether numpy is importable. If numpy is missing, run the third cell to install it into this kernel.

In [ ]:
import sys, subprocess
print('Notebook sys.executable:', sys.executable)
try:
    import numpy as np
    print('Imported numpy, version:', np.__version__)
except Exception as e:
    print('Import numpy failed:', repr(e))
print('\npip show numpy:')
subprocess.run([sys.executable, '-m', 'pip', 'show', 'numpy'])


Notebook sys.executable: c:\Users\ADMIN\OneDrive\Desktop\RP Model\venv\Scripts\python.exe


In [5]:
# Run this cell only if numpy is missing in the kernel above.
import sys, subprocess
print('Installing numpy into:', sys.executable)
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'numpy'])


Installing numpy into: c:\Users\ADMIN\OneDrive\Desktop\RP Model\venv\Scripts\python.exe


0

In [4]:
import numpy as np
import pandas as pd

np.random.seed(42)

n = 200

RBC = np.random.uniform(4.5, 8.5, n)
WBC = np.random.uniform(5, 15, n)
HB  = np.random.uniform(8, 18, n)
Platelets = np.random.uniform(150, 600, n)
Creatinine = np.random.uniform(0.5, 2.0, n)
Glucose    = np.random.uniform(60, 160, n)
Dose = np.random.uniform(0.3, 1.3, n)

Outcome = ((HB > 12) & (Creatinine < 1.2) & (Dose < 1.0)).astype(int)

df = pd.DataFrame({
    "RBC": RBC,
    "WBC": WBC,
    "HB": HB,
    "Platelets": Platelets,
    "Creatinine": Creatinine,
    "Glucose": Glucose,
    "Dose": Dose,
    "Outcome": Outcome
})

df.head()


ModuleNotFoundError: No module named 'numpy'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)

y_pred = logreg.predict(X_test_scaled)

accuracy_log = accuracy_score(y_test, y_pred)
accuracy_log


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

accuracy_rf = accuracy_score(y_test, rf_pred)
accuracy_rf


In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=4)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

accuracy_xgb = accuracy_score(y_test, xgb_pred)
accuracy_xgb
